In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

### Set Up Environment

In [ ]:
connections = {
"mainnet": {"iconservice": "https://ctz.solidwallet.io",       "nid": 1},
"yeouido": {"iconservice": "https://bicon.net.solidwallet.io", "nid": 3},
"euljiro": {"iconservice": "https://test-ctz.solidwallet.io",  "nid": 2},
"pagoda":  {"iconservice": "https://zicon.net.solidwallet.io", "nid":80},
"custom":  {"iconservice": "http://13.57.193.25:9000/",        "nid": 3}}

env = connections["custom"] # mainnet, yeouido, euljiro, pagoda, custom

In [ ]:
from iconsdk.exception import JSONRPCException
from iconsdk.libs.in_memory_zip import gen_deploy_data_content
from iconsdk.icon_service import IconService
from iconsdk.providers.http_provider import HTTPProvider
from iconsdk.builder.transaction_builder import CallTransactionBuilder, TransactionBuilder, DeployTransactionBuilder
from iconsdk.builder.call_builder import CallBuilder
from iconsdk.signed_transaction import SignedTransaction
from iconsdk.wallet.wallet import KeyWallet
from iconsdk.utils.convert_type import convert_hex_str_to_int
from repeater import retry
import json
import os

ICX = 1000000000000000000 # 10**18
GOVERNANCE_ADDRESS = "cx0000000000000000000000000000000000000000"

contracts = {'dex':  {'zip': 'core_contracts/dex.zip',   'SCORE': 'cx7915f8111043cb6a8f42a528af16de0fe0690834'},
             'sicx': {'zip': 'token_contracts/sicx.zip', 'SCORE': 'cxced84c6b7d71465a9b99396137df2d7da233a6a3'},
             'icd':  {'zip': 'token_contracts/icd.zip',  'SCORE': 'cx9e7d7d88de153f7ae3117472f278169475674b09'},
             'bal':  {'zip': 'token_contracts/bal.zip',  'SCORE': 'cx7ddc34e80e6e96f66f67255fa8f3b87e6b7a648a'}}

@retry(JSONRPCException, tries=10, delay=1, back_off=2)
def get_tx_result(_tx_hash):
    tx_result = icon_service.get_transaction_result(_tx_hash)
    return tx_result

In [ ]:
icon_service = IconService(HTTPProvider(env["iconservice"], 3))
NID = env["nid"]

In [ ]:
wallet = KeyWallet.load("keystores/keystore_test1.json", "test1_Account")

In [ ]:
step_limit = 4000100000

for score, loc in contracts.items():
    deploy_transaction = DeployTransactionBuilder()\
        .from_(wallet.get_address())\
        .to(GOVERNANCE_ADDRESS)\
        .nid(NID)\
        .nonce(100)\
        .content_type("application/zip")\
        .content(gen_deploy_data_content(loc["zip"]))\
        .build()

    signed_transaction = SignedTransaction(deploy_transaction, wallet, step_limit)
    tx_hash = icon_service.send_transaction(signed_transaction)

    res = get_tx_result(tx_hash)
    contracts[score]["SCORE"] = res['scoreAddress']
contracts